In [ ]:

!pip install torch transformers accelerate sentencepiece bitsandbytes


In [ ]:

from dataclasses import dataclass
import re
from typing import Optional, Tuple
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


In [ ]:

@dataclass
class ModelCfg:
    model_name: str = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    device_map: str = "auto"
    torch_dtype: str = "bfloat16"
    load_in_4bit: bool = False
    max_new_tokens: int = 256
    temperature: float = 0.0
    top_p: float = 1.0

class Llama31Judge:
    def __init__(self, cfg: ModelCfg = ModelCfg()):
        quant_args = {}
        if cfg.load_in_4bit:
            quant_args = dict(load_in_4bit=True, bnb_4bit_compute_dtype=getattr(torch, cfg.torch_dtype))
        self.tok = AutoTokenizer.from_pretrained(cfg.model_name, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            cfg.model_name,
            device_map=cfg.device_map,
            torch_dtype=getattr(torch, cfg.torch_dtype),
            **quant_args
        )
        self.gen_cfg = GenerationConfig(
            max_new_tokens=cfg.max_new_tokens,
            temperature=cfg.temperature,
            top_p=cfg.top_p,
            do_sample=cfg.temperature > 0
        )

    @torch.inference_mode()
    def generate(self, prompt: str) -> str:
        inputs = self.tok(prompt, return_tensors="pt").to(self.model.device)
        out = self.model.generate(**inputs, generation_config=self.gen_cfg)
        text = self.tok.decode(out[0], skip_special_tokens=True)
        return text[len(self.tok.decode(inputs["input_ids"][0], skip_special_tokens=True)) :].strip()


In [ ]:

SHARED_SAMPLING_PROMPT = """Read the question, analyze step by step, provide your answer and your confidence in this
answer. Note: The confidence indicates how likely you think your answer is true.
If you are uncertain about your answer to the question, convey this uncertainty linguistically
by precisely hedging this answer. Use the following format to answer:
Explanation: [insert short step-by-step analysis here]
Answer: [ONLY the option LETTER, where the LETTER is one of {Letters}]
Confidence (0-100): [Your confidence level, please only include the numerical number in the
range of 0-100]%

Only give me the reply according to this format, don’t give me any other words.

{Question}
"""

LVU_JUDGE_PROMPT = """You will be given a question and a proposed answer generated by a language model. Your
task is to output an confidence score: a number in [0,1] representing how certain the language
model is about the truthfulness of its generated answer, judged solely based on the provided
answer. You should pay special attention to the usage of any hedging modifiers in the original
answer, used to convey uncertainty in the truthfulness of the answer. A score close to 1.0
indicates that the model is very certain about the truthfulness answer. And a score close to
0.0 means that the model is highly uncertain, and the answer likely contains some hedging expressions. If the proposed answer punts the question, the confidence score should be 0.0.

Here are the question and proposed answer to be judged:
Question:
{Question}
Proposed Answer:
{Response}

Provide your response only in the following format:
Confidence score: [confidence score (0-1)].
"""


In [ ]:

ANSWER_REGEXES = [
    r"[Aa]nswer:?[\s]*[
]*([A-J])",
    r"[Aa]nswer:[\s]*[
]*\(?([A-J])\)?",
    r"[Aa]nswer:[\s]*[
]*\[?([A-J])\]?",
    r"[Aa]nswer:[\s]*[
]*([A-J])[,)]",
    r"[Aa]nswer:[\s]*[
]*([A-J])\s*,?.*",
    r"Answer:\n([A-J])\nConfidence",
    r"answer is\s*\[?\(?([A-J])\]?\)?",
    r"answer should be\s*\[?\(?([A-J])\]?\)?",
    r"best option is \(?([A-J])\)?",
    r"best match is option \(?([A-J])\)?",
    r"the closest is \(?([A-J])\)?",
    r"Answer:\n*^([A-J])$",
    r"^([A-J])$",
]
CONF_STRIP_REGEXES = [
    r"[Cc]onfidence\s*\(0-100\):\s*[\(]?[\[]?(\d+)[\)]?[\]]?%?",
    r"[Cc]onfidence[:]?:\s*(\d+)%?",
    r"[Cc]onfidence [\(0-100\)]?:\s*\[(\d+)%?\]",
    r"[Cc]onfidence [Ll]evel\s*\(0-100\):\s*(\d+)%?",
    r"[Cc]onfidence [Ll]evel[:]?:\s*(\d+)%?",
    r"[Cc]onfidence [Ll]evel[\(0-100\)]?:\s*\[(\d+)%?\]",
    r"[Cc]onfidence \(100\):\s*\w*,\s*(\d+)%?",
    r"[Cc]onfidence\s*\(\d+\)\s*:\s*(\d+)%?",
    r"[Cc]onfidence\s*[\(]?(\d+)[\)]?%?",
]


In [ ]:

def extract_letter(response_text: str) -> Optional[str]:
    for pat in ANSWER_REGEXES:
        m = re.search(pat, response_text, flags=re.MULTILINE)
        if m:
            return m.group(1).strip()
    return None

def strip_numeric_confidence(response_text: str) -> Tuple[str, Optional[int]]:
    conf_val = None
    cleaned = response_text
    for pat in CONF_STRIP_REGEXES:
        m = re.search(pat, cleaned, flags=re.MULTILINE)
        if m:
            try:
                conf_val = int(m.group(1))
            except Exception:
                pass
            cleaned = re.sub(pat, "", cleaned)
    return cleaned.strip(), conf_val

def parse_judge_confidence(judge_output: str) -> Optional[float]:
    m = re.search(r"Confidence score:\s*([01](?:\.\d+)?)", judge_output)
    return float(m.group(1)) if m else None


In [ ]:

cfg = ModelCfg(load_in_8bit=True)  
mdl = Llama31Judge(cfg)

# Example MMLU-style question
q = "Which gas is most responsible for the greenhouse effect on Earth?\nA) Oxygen\nB) Nitrogen\nC) Carbon dioxide\nD) Argon"
letters = "ABCD"

# 1) Sample response
resp = mdl.generate(SHARED_SAMPLING_PROMPT.format(Letters=letters, Question=q))

# 2) Extract answer letter
ans = extract_letter(resp)

# 3) Remove numeric confidence before LVU judging
cleaned, nvu = strip_numeric_confidence(resp)

# 4) Judge LVU confidence
judge_prompt = LVU_JUDGE_PROMPT.format(Question=q, Response=cleaned)
judge_out = mdl.generate(judge_prompt)
lvu = parse_judge_confidence(judge_out)

print("RAW RESPONSE:\n", resp)
print("Predicted Letter:", ans)
print("Numeric Verbal Confidence:", nvu)
print("LVU Judge Output:", judge_out)
print("LVU Confidence [0-1]:", lvu)
